In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Introduccion a Machine Learning II

---

En esta práctica vamos a poner en práctica los conceptos de aprendizaje supervisado y no supervisado que estuvimos viendo.

### Aprendizaje supervisado

---

El ejercicio será entrenar un modelo de Naive Bayes Gaussiano para clasificar distintas especies de flores. Trabajaremos sobre un dataset típico en _machine learning_ conocido como [Iris](https://es.wikipedia.org/wiki/Conjunto_de_datos_flor_iris), el cual provee información sobre 150 muestras de este tipo de flor. Contamos con estas características: ancho y largo de sépalos y pétalos y *la etiqueta de la especie a la que corresponde cada muestra (Setosa, Versicolor y Virginica)*.

<img src="https://shahinrostami.com/images/ml-with-kaggle/iris-2338142_960_720-1.jpg">
<center><i>La flor iris</i></center>

Comencemos cargando el dataset, inspeccionando la tabla y visualizando los datos:

In [ ]:
import seaborn as sns

iris = sns.load_dataset('iris')
iris.sample(5)

In [ ]:
iris.describe()

In [ ]:
sns.pairplot(iris, hue='species');

<div id="caja10" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/para_seguir_pensando.png" style="align:left"/> </div>
  <br>
  <br>
  <div style="float:left;width: 85%;"><label><b>¿Qué destacamos de estas visualizaciones? ¿Existen variables que permitan distinguir entre especies?</b></label></div>
</div>

Recordemos los pasos del _workflow_ típico de Scikit-Learn:

1. Seleccionar una clase de modelo
2. Elegir los hiperparámetros del modelo
3. Preparar los datos en una matriz de _features_ y un vector _target_
4. Separar los sets de entrenamiento y de testeo
5. Ajustar el modelo a los datos de entrenamiento
6. Predecir etiquetas para datos desconocidos
7. Evaluar la _performance_ del modelo

##### 1. Seleccionar una clase de modelo

In [ ]:
from sklearn.naive_bayes import GaussianNB

##### 2. Elegir los hiperparámetros del modelo

In [ ]:
model = GaussianNB()

##### 3. Preparar los datos en una matriz de _features_ y un vector _target_

In [ ]:
iris.sample(2)

In [ ]:
# ¿Cuáles son las features y cuál es el target en este caso?
X = iris.drop('species', axis=1)
y = iris['species']

##### 4. Separar los sets de entrenamiento y de testeo

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)

##### 5. Ajustar el modelo a los datos de entrenamiento

In [ ]:
model.fit(Xtrain, ytrain)

##### 6. Predecir etiquetas para datos desconocidos

In [ ]:
ypred = model.predict(Xtest)

print('Original: ',list(ytest[0:3]))
print('Predicho: ',ypred[0:3])

##### 7. Evaluar la _performance_ del modelo

$$ \text{Accuracy}=\frac{\text{predicciones correctas}}{\text{casos totales}} $$

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest, ypred)

### Aprendizaje no supervisado

---

El ejercicio será entrenar un modelo no supervisado, usando un algoritmo de clustering.

Los algoritmos de _clustering_ nos permiten segmentar las observaciones asignándolas automáticamente a distintos grupos o _clusters_, de forma de que observaciones *similares* se encuentren agrupadas y que los grupos formados sean más bien *distintos entre sí*. Recordemos que el dataset no tiene una *variable target*.

Comencemos cargando y analizando el dataset de la biblioteca Seaborn con datos de los geysers.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

geyser = sns.load_dataset('geyser')
geyser.sample(5)

Si visualizamos los datos, vemos que la *variable kind* separa en dos grupos separados los datos. Vamos a comprobar que haciendo clustering obtenemos el mismo resultado.

In [ ]:
sns.set_style('whitegrid')
sns.relplot(data=geyser, x='duration', y='waiting', hue=geyser.kind.tolist());

*Notemos que, pese a tratarse de un problema no supervisado, el paso a paso es prácticamente igual al usado con los modelos supervisados.*

##### 1. Seleccionamos una clase de modelo

Vamos a aplicar una técnica clásica de _clustering_ llamada K-Means.

In [ ]:
from sklearn.cluster import KMeans

##### 2. Elegir los hiperparámetros del modelo

n_clusters define *cuántos clusters queremos obtener* y random_state controla la aleatoriedad inicial.

In [ ]:
model = KMeans(n_clusters=2, random_state=0)

##### 3. Preparar los datos en una matriz de _features_

Como se trata de un problema no supervisado, no contaremos con una variable objetivo. Por eso, construimos la matriz de _features_ descartando la variable *kind*.

In [ ]:
X_geyser = geyser.drop('kind', axis=1)
X_geyser.shape

Dado que K-Means es un algoritmo que trabaja *calculando distancias*, un paso previo al ajuste del modelo consiste en **estandarizar los valores**, de forma de eliminar posibles inconvenientes asociados a la escala de las distintas variables.

La clase **`StandardScaler()`**, transforma los datos, para que las variables tengan *media 0 y desvío estándar 1*.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

Los transformadores también se ajustan a los datos, al igual que los estimadores:

In [ ]:
scaler.fit(X_geyser)

A diferencia de los estimadores, en lugar del método `predict()`, los transformadores tienen el método `transform()`:

In [ ]:
X = scaler.transform(X_geyser)

In [ ]:
X[:3]

In [ ]:
X_geyser[:3]

##### 4. Separar los sets de entrenamiento y de testeo

En los problemas de _clustering_ (y de aprendizaje no supervisado en general) *no es necesario hacer la separación de los conjuntos de entrenamiento y testeo.*

##### 5. Ajustar el modelo a los datos

In [ ]:
model.fit(X)

##### 6. Predecir etiquetas

En este caso, la predicción consiste en generar etiquetas que identifiquen cada observación con un _cluster_ en particular.

Como seleccionamos k=2 clusters, las etiquetas son 0 y 1.

In [ ]:
y_km = model.predict(X)
y_km[:5]

##### 7. Evaluar la _performance_ del modelo


Grafiquemos ahora un scatterplot con las dos variables originales *duration* y *waiting* , pero agregando como tercera dimensión al *cluster asignado*.

Observamos que también separa las observaciones en dos grupos separados, con mínimas diferencias respecto al valor original de la variable *kind*.

Existen métodos más formales para evaluar la performance del modelo, que veremos en las próximas clases.

In [ ]:
X_geyser['cluster'] = y_km
sns.set_style('whitegrid')
sns.relplot(data=X_geyser, x='duration', y='waiting', hue=X_geyser.cluster.tolist());